In [3]:
import csv
import json
import sys
import pandas as pd
from pathlib import Path
import requests
import pprint
sys.path.insert(0, 'C:/Users/sam/Desktop/ScrapeTest') 
from cookie_monster import COOKIE
import demjson
pd.set_option("display.max_rows", None, "display.max_columns", None)

user_cookie = dict()

for item in COOKIE.split("; "):
    k, v = item.split("=")
    user_cookie[k] = v

full_headers_dict = {"summonerName":"player", "championId": "champion", "magicDamageDealt":"totalmagicdamagedealt", "physicalDamageDealt":"totalphysicaldamagedealt", "physicalDamageTaken":"physicaldamagetaken", "kills":"kills","deaths":"deaths","assists":"assists", 'firstBloodKill': 'firstblood', 'totalDamageDealtToChampions': 'totaldamagetochampions', 'physicalDamageDealtToChampions': 'physicaldamagetochampions', 'magicDamageDealtToChampions': 'magicdamagetochampions', 'trueDamageDealtToChampions': 'truedamagetochampions', 'damageDealtToObjectives': 'totaldamagetoobjectives', 'damageDealtToTurrets': 'totaldamagetoturrets', 'totalHeal': 'damagehealed', 'totalDamageTaken': 'damagetaken', 'magicalDamageTaken': 'magicdamagetaken', 'wardsKilled': 'wardsdestroyed', 'sightWardsBoughtInGame': 'stealthwardspurchased', 'visionWardsBoughtInGame': 'controlwardspurchased', 'totalMinionsKilled': 'minionskilled', 'neutralMinionsKilledTeamJungle': "neutralminionskilledinteam'sjungle", 'neutralMinionsKilledEnemyJungle': 'neutralminionskilledinenemyjungle', 'killingSprees': 'killingsprees', 'longestTimeSpentLiving': 'longesttimespentliving', 'damageSelfMitigated': 'selfmitigateddamage', 'visionScore': 'visionscore', 'timeCCingOthers': 'timeccingothers', 'turretKills': 'totalTurretKills', 'inhibitorKills': 'totalInhibitorKills', 'totalTimeCrowdControlDealt': 'totaltimeapplyingcc', 'firstBloodAssist': 'firstbloodassist', 'firstTowerKill': 'firsttowerkill', 'firstTowerAssist': 'firsttowerassist', 'firstInhibitorKill': 'firstinhibkill', 'firstInhibitorAssist': 'firstinhibassist', 'doubleKills': 'doublekills', 'tripleKills': 'triplekills', 'quadraKills': 'quadrakills', 'pentaKills': 'pentakills', 'goldEarned':'goldearned', 'goldSpent': 'goldspent', "largestCriticalStrike":"largestcriticalstrike", "largestKillingSpree":"largestkillingspree", "largestMultiKill":"largestmultikill", "teamId":'side', 'gameId':'gameId'}
lpl_full_headers_dict = {"name": "player","hero":"champion","kill":"kills","death":"deaths","assist":"assists",'firstBlood': 'firstblood', 'totalDamageToChamp': 'totaldamagetochampions', 'pDamageToChamp': 'physicaldamagetochampions', 'mDamageDealtToChamp': 'magicdamagetochampions', 'trueDamageDealtToChampions': 'truedamagetochampions', 'damageDealtToObjectives': 'totaldamagetoobjectives', 'damageDealtToTurrets': 'totaldamagetoturrets', 'totalHeal': 'damagehealed', 'totalDamageTaken': 'damagetaken', 'mDamageTaken': 'magicdamagetaken', 'wardsKilled': 'wardsdestroyed', 'sightWardsBoughtInGame': 'stealthwardspurchased', 'visionWardsBought': 'controlwardspurchased', 'lasthit': 'minionskilled', 'neutralKilledTJungle': "neutralminionskilledinteam'sjungle", 'neutralKilledEJungle': 'neutralminionskilledinenemyjungle', 'killingSprees': 'killingsprees', 'longestTimeSpentLiving': 'longesttimespentliving', 'damageSelfMitigated': 'selfmitigateddamage', 'visionScore': 'visionscore', 'timeCCingOthers': 'timeccingothers', 'towerKills': 'totalTurretKills', 'inhibitorKills': 'totalInhibitorKills', 'totalTimeCrowdControlDealt': 'totaltimeapplyingcc', 'firstBloodAssist': 'firstbloodassist', 'firstTowerKill': 'firsttowerkill', 'firstTowerAssist': 'firsttowerassist', 'firstInhibitorKill': 'firstinhibkill', 'firstInhibitorAssist': 'firstinhibassist', 'dKills': 'doublekills', 'tKills': 'triplekills', 'qKills': 'quadrakills', 'pKills': 'pentakills', 'side':'side', 'gold':'goldearned', 'GoldSpent': 'goldspent', "pDamageDealt":"totalphysicaldamagedealt", "pDamageTaken":"physicaldamagetaken", 'mDamageDealt':'totalmagicdamagedealt', 'largestCriticalStrike':'largestcriticalstrike',  "largestKillingSpree":"largestkillingspree", "largestMultiKill":"largestmultikill", 'game-id':'gameId'}
scrape_headers = full_headers_dict.keys()
lpl_scrape_headers = lpl_full_headers_dict.keys()
table_headers = full_headers_dict.values()

#This is pulling the champion data.
champ_dd = requests.get("https://ddragon.leagueoflegends.com/cdn/11.8.1/data/en_US/champion.json", cookies=user_cookie)
champ_json = champ_dd.json()
champ_name = champ_json["data"] #this creates a list of all the champions
champ_key_dict = {}
for i in champ_name:
    champ_key_dict.update({champ_json["data"][i]["key"]:i})

raw_url_list = ["https://matchhistory.euw.leagueoflegends.com/en/#match-details/EUW1/5339657139/2134428503602144?tab=overview",
"https://matchhistory.na.leagueoflegends.com/en/#match-details/NA1/2957580992/215480663?tab=overview"]

#"https://matchhistory.br.leagueoflegends.com/pt/#match-details/ESPORTSTMNT03/570139?gameHash=ba727c1db6d1cfbb&amp;tab=overview",

def amateur_url_for_request_scraping(raw_url):
    if "/NA1/" in raw_url:
        cut = raw_url.split("#match-details/")[1].split("/")[:-1]
        end = raw_url.split("/")[-1].split("?")[0]
        new_url = "https://acs.leagueoflegends.com/v1/stats/game/" + cut[0] + "/" +  cut[1] + "?visiblePlatformId=NA1&visibleAccountId=" + end
        return (new_url)
    elif "/EUW1/" in raw_url:
        cut = raw_url.split("#match-details/")[1].split("/")[:-1]
        end = raw_url.split("/")[-1].split("?")[0]
        new_url = "https://acs.leagueoflegends.com/v1/stats/game/" + cut[0] + "/" +  cut[1] + "?visiblePlatformId=EUW1&visibleAccountId=" + end
        return (new_url)

def amateur_get_match_data(url, test):
    good_url = amateur_url_for_request_scraping(url)
    json_file = requests.get(good_url, cookies=user_cookie)
    assert json_file.status_code == 200
    json_content = json_file.json()
    if test == True:
        pprint.pprint(json_content)
    return(json_content)

def amateur_build_dataframe(input_match_data, test):
    json_content = input_match_data
    count = 0
    while count < 10:
        #player = json_content["participantIdentities"][count]["player"]["summonerName"]
        role = y['participants'][count]['timeline']['role']
        lane = y['participants'][count]['timeline']['lane']
        if role:
            player = lane +", " + role
        else:
            player =  lane
        champion_id = json_content["participants"][count]["championId"]
        champion_name = champ_key_dict[str(champion_id)]
        gameId = json_content["gameId"]
        stats = json_content["participants"][count]["stats"]
        stats.update({"championId": champion_name , "gameId": gameId, "summonerName":player})
        if count == 0:
            df = pd.DataFrame([stats])
            count = count + 1
        else:
            new_df = pd.DataFrame([stats])
            count = count + 1
            df = df.append(new_df) 
    if test == True:
        pprint.pprint(df)
    return(df)

def amateur_fix_dataframe(dataframe):
    positions = {"TOP, SOLO":"top", "MIDDLE, SOLO":"mid", "JUNGLE, NONE":"jng", "BOTTOM, DUO_CARRY":"bot", "BOTTOM, DUO_SUPPORT":"sup"}
    dataframe["summonerName"] = dataframe["summonerName"].apply(lambda x: positions[x])
    dataframe["win"] = dataframe["summonerName"].apply(lambda x: 1 if x else 0)
    return (dataframe)


#for i in raw_url_list:
#x = amateur_url_for_request_scraping(raw_url_list[0])
#print(x)
y = amateur_get_match_data(raw_url_list[0], False)
z = amateur_build_dataframe(y, False)
aa = amateur_fix_dataframe(z)
display(aa)

#https://acs.leagueoflegends.com/v1/stats/game/NA1/2957580992?visiblePlatformId=NA1&visibleAccountId=215480663
#"https://matchhistory.na.leagueoflegends.com/en/#match-details/NA1/2957580992/215480663?tab=overview"

,participantId,win,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,inhibitorKills,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,wardsPlaced,wardsKilled,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,combatPlayerScore,objectivePlayerScore,totalPlayerScore,totalScoreRank,playerScore0,playerScore1,playerScore2,playerScore3,playerScore4,playerScore5,playerScore6,playerScore7,playerScore8,playerScore9,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,championId,gameId,summonerName
0,1,1,3074,3078,3053,3123,3047,1033,3340,3,6,4,3,1,1,948,0,0,0,0,0,179707,10310,159260,10137,0,27525,7440,16912,3173,10512,1,22695,8329,4212,15,12,31048,17005,13754,288,12662,12658,1,0,224,8,5,0,229,16,1,0,9,0,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8437,1157,799,0,8446,1368,0,0,8429,63,15,13,8451,204,0,0,8321,3,0,0,8352,280,1687,950,8400,8300,5008,5008,5003,Gangplank,5339657139,top
0,2,1,6672,1018,3035,3031,3006,3046,3363,3,3,8,2,1,1,1039,0,0,0,0,0,212023,19477,176641,15904,596,18993,5048,12497,1446,3301,4,9604,20693,3044,36,0,14213,4571,9366,275,14367,13800,2,0,266,28,24,0,51,16,4,0,12,5,True,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9923,43,79,0,8139,934,0,0,8138,18,0,0,8106,5,0,0,9103,21,10,0,9111,625,220,0,8100,8000,5005,5008,5002,Kaisa,5339657139,bot
0,3,1,6632,3123,3065,0,3158,3110,3364,3,4,8,3,1,1,758,0,0,0,0,0,198625,22632,163888,12103,0,13381,2114,10071,1195,16870,1,36538,23430,0,24,15,33914,8950,23794,1170,11569,11400,0,0,19,177,128,15,508,16,6,0,7,4,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8010,198,0,0,9111,610,220,0,9105,13,30,0,8299,286,0,0,8429,63,20,14,8453,1575,0,0,8000,8400,5005,5008,5002,Hecarim,5339657139,jng
0,4,1,6691,6333,2055,3857,6695,3117,3364,8,5,5,4,1,2,756,0,0,0,0,0,22622,0,13053,9568,0,7956,0,5082,2873,4806,1,10356,2376,471,75,10,19164,8428,10525,210,12081,11350,0,0,33,0,0,0,63,12,10,0,37,8,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9923,14,51,0,8126,231,0,0,8138,18,0,0,8106,5,0,0,8444,830,0,0,8242,65,0,0,8100,8400,5008,5008,5003,Pyke,5339657139,sup
0,5,1,3020,3152,3157,2420,3916,3041,3363,6,6,6,2,2,2,598,1,0,0,0,0,119461,109139,9697,625,0,20042,18476,941,625,2142,1,12455,2253,0,29,1,21089,6625,14076,387,11652,11175,0,0,178,4,0,0,76,16,3,0,12,0,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8010,244,0,0,8009,609,0,0,9105,14,30,0,8299,679,0,0,8444,954,0,0,8451,197,0,0,8000,8400,5008,5008,5002,Akali,5339657139,mid
0,6,1,3152,1056,1082,4629,3158,3089,3363,7,2,3,4,3,2,914,1,1,0,0,0,197404,180617,16781,5,0,23327,22294,1028,5,28721,1,12866,12464,6658,21,2,27444,6538,19601,1304,14464,11750,3,0,249,16,9,4,92,17,0,0,11,2,False,False,False,False,False,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8230,13,0,0,8275,8,0,0,8210,13,0,0,8236,48,0,0,8444,1524,0,0,8451,259,0,0,8200,8400,5007,5008,5002,Vladimir,5339657139,top
0,7,1,6631,3047,3053,1011,3076,1028,3364,1,6,16,0,1,0,489,0,0,0,0,0,186611,17219,154850,14542,2,15680,1005,13711,963,15264,1,25712,26850,2489,40,18,31

In [63]:
a = y.keys()
count = 0
while count < 10:
    role = y['participants'][count]['timeline']['role']
    lane = y['participants'][count]['timeline']['lane']
    print (role, lane)
    count = count + 1


#y[]

SOLO TOP
DUO_CARRY BOTTOM
NONE JUNGLE
DUO_SUPPORT BOTTOM
SOLO MIDDLE
SOLO TOP
NONE JUNGLE
SOLO MIDDLE
DUO_CARRY BOTTOM
DUO_SUPPORT BOTTOM


In [62]:
role = y['participants'][count]['timeline']['role']

'SOLO'

In [67]:
raw_url_list = ["https://matchhistory.euw.leagueoflegends.com/en/#match-details/EUW1/5339657139/2134428503602144?tab=overview",
"https://matchhistory.na.leagueoflegends.com/en/#match-details/NA1/2957580992/215480663?tab=overview", "https://matchhistory.br.leagueoflegends.com/pt/#match-details/ESPORTSTMNT03/570139?gameHash=ba727c1db6d1cfbb&amp;tab=overview",
]

for i in raw_url_list:
    if "ESPORTS" in i:
        print('pro')
    else: print ('amateur')


amateur
amateur
pro


In [18]:
import csv
import json
import sys
import pandas as pd
from pathlib import Path
import requests
import pprint
sys.path.insert(0, 'C:/Users/sam/Desktop/ScrapeTest') 
from cookie_monster import COOKIE
import demjson
pd.set_option("display.max_rows", None, "display.max_columns", None)

def get_urllist():
    l = []
    url_file = r"C:/Users/sam/Desktop/ScrapeTest/all_urls2.csv"
    file = pd.read_csv(url_file, header=0)
    l = list(file.url)
    print (l[0])
    single = list(set(l))
    print(len(single))
    return (single)

def url_for_request_scraping(raw_url):
    cut = raw_url.split("#match-details/")[1].split("&amp;tab=overview,Unnamed: 31False")[0]
    new_url = "https://acs.leagueoflegends.com/v1/stats/game/"+cut
    return (new_url)

raw_url = "http://matchhistory.na.leagueoflegends.com/en/#match-details/TRLH1/1002280274?gameHash=d6b1888cdae97397&amp;tab=overview"

x = url_for_request_scraping(raw_url)
print(x)

https://acs.leagueoflegends.com/v1/stats/game/TRLH1/1002280274?gameHash=d6b1888cdae97397&amp;tab=overview


In [9]:
#List of Issues:
#lpl Expecting value: line 1 column 1 (char 0)
#non-lpl 'summonerName' -> Frequent
#non-lpl list index out of range


SyntaxError: invalid syntax (Temp/ipykernel_5220/959021487.py, line 5)

In [19]:
import sys

In [27]:
try:
    print 'dummy'
    print("thick")
except Exception as e:
    print(e, "Error on line {}".format(sys.exec_info()[-1].tb_lineno)


SyntaxError: Missing parentheses in call to 'print'. Did you mean print('dummy')? (Temp/ipykernel_16244/542682145.py, line 2)

In [6]:
import requests
import re
import sys
import os
from requests.auth import HTTPBasicAuth
import time
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import pprint
from pathlib import Path #https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f
import csv
import json
import traceback
sys.path.insert(0, 'C:/Users/sam/Desktop/ScrapeTest')
from cookie_monster import COOKIE

###Basically, key = scraped value, value = table output value
full_headers_dict = {"gameVersion":"patch", "summonerName":"player", "championId": "champion", "magicDamageDealt":"totalmagicdamagedealt", "physicalDamageDealt":"totalphysicaldamagedealt", "physicalDamageTaken":"physicaldamagetaken", "kills":"kills","deaths":"deaths","assists":"assists", 'firstBloodKill': 'firstblood', 'totalDamageDealtToChampions': 'totaldamagetochampions', 'physicalDamageDealtToChampions': 'physicaldamagetochampions', 'magicDamageDealtToChampions': 'magicdamagetochampions', 'trueDamageDealtToChampions': 'truedamagetochampions', 'damageDealtToObjectives': 'totaldamagetoobjectives', 'damageDealtToTurrets': 'totaldamagetoturrets', 'totalHeal': 'damagehealed', 'totalDamageTaken': 'damagetaken', 'magicalDamageTaken': 'magicdamagetaken', 'wardsKilled': 'wardsdestroyed', 'sightWardsBoughtInGame': 'stealthwardspurchased', 'visionWardsBoughtInGame': 'controlwardspurchased', 'totalMinionsKilled': 'minionskilled', 'neutralMinionsKilledTeamJungle': "neutralminionskilledinteam'sjungle", 'neutralMinionsKilledEnemyJungle': 'neutralminionskilledinenemyjungle', 'killingSprees': 'killingsprees', 'longestTimeSpentLiving': 'longesttimespentliving', 'damageSelfMitigated': 'selfmitigateddamage', 'visionScore': 'visionscore', 'timeCCingOthers': 'timeccingothers', 'turretKills': 'totalTurretKills', 'inhibitorKills': 'totalInhibitorKills', 'totalTimeCrowdControlDealt': 'totaltimeapplyingcc', 'firstBloodAssist': 'firstbloodassist', 'firstTowerKill': 'firsttowerkill', 'firstTowerAssist': 'firsttowerassist', 'firstInhibitorKill': 'firstinhibkill', 'doubleKills': 'doublekills', 'tripleKills': 'triplekills', 'quadraKills': 'quadrakills', 'pentaKills': 'pentakills', 'goldEarned':'goldearned', 'goldSpent': 'goldspent', "largestCriticalStrike":"largestcriticalstrike", "largestKillingSpree":"largestkillingspree", "largestMultiKill":"largestmultikill", "teamId":'side', 'gameId':'gameid'}
lpl_full_headers_dict = {"name": "player","hero":"champion","kill":"kills","death":"deaths","assist":"assists",'firstBlood': 'firstblood', 'totalDamageToChamp': 'totaldamagetochampions', 'pDamageToChamp': 'physicaldamagetochampions', 'mDamageDealtToChamp': 'magicdamagetochampions', 'trueDamageDealtToChampions': 'truedamagetochampions', 'damageDealtToObjectives': 'totaldamagetoobjectives', 'damageDealtToTurrets': 'totaldamagetoturrets', 'totalHeal': 'damagehealed', 'totalDamageTaken': 'damagetaken', 'mDamageTaken': 'magicdamagetaken', 'wardsKilled': 'wardsdestroyed', 'sightWardsBoughtInGame': 'stealthwardspurchased', 'visionWardsBought': 'controlwardspurchased', 'lasthit': 'minionskilled', 'neutralKilledTJungle': "neutralminionskilledinteam'sjungle", 'neutralKilledEJungle': 'neutralminionskilledinenemyjungle', 'killingSprees': 'killingsprees', 'longestTimeSpentLiving': 'longesttimespentliving', 'damageSelfMitigated': 'selfmitigateddamage', 'visionScore': 'visionscore', 'timeCCingOthers': 'timeccingothers', 'towerKills': 'totalTurretKills', 'inhibitorKills': 'totalInhibitorKills', 'totalTimeCrowdControlDealt': 'totaltimeapplyingcc', 'firstBloodAssist': 'firstbloodassist', 'firstTowerKill': 'firsttowerkill', 'firstTowerAssist': 'firsttowerassist', 'firstInhibitorKill': 'firstinhibkill', 'firstInhibitorAssist': 'firstinhibassist', 'dKills': 'doublekills', 'tKills': 'triplekills', 'qKills': 'quadrakills', 'pKills': 'pentakills', 'side':'side', 'gold':'goldearned', 'GoldSpent': 'goldspent', "pDamageDealt":"totalphysicaldamagedealt", "pDamageTaken":"physicaldamagetaken", 'mDamageDealt':'totalmagicdamagedealt', 'largestCriticalStrike':'largestcriticalstrike',  "largestKillingSpree":"largestkillingspree", "largestMultiKill":"largestmultikill", 'gameId':'gameid'}
scrape_headers = full_headers_dict.keys()
lpl_scrape_headers = lpl_full_headers_dict.keys()
table_headers = full_headers_dict.values()
reg = r'^([\w]+.[\w]+)'

#Function that grabs the URL list from a csv
def get_urllist(csv):
    l = []
    #url_file = r"F:/LeagueStats/scraping/MatchHistoryScraping/data/URL.csv"
    url_file = csv
    file = pd.read_csv(url_file, header=0)
    l = list(file.url)
    single = list(set(l))
    for i in single:
        if type(i) != str:
            single.remove(i)
    print(len(single))
    return (single)

def url_for_request_scraping(raw_url):
    cut = raw_url.split("#match-details/")[1].split("&amp;tab=overview,Unnamed: 31False")[0]
    new_url = "https://acs.leagueoflegends.com/v1/stats/game/"+cut
    return (new_url)

# Format your cookie into a dictionary form
user_cookie = dict()
for item in COOKIE.split("; "):
    k, v = item.split("=")
    user_cookie[k] = v

#This is pulling the champion data.
champ_dd = requests.get("https://ddragon.leagueoflegends.com/cdn/11.8.1/data/en_US/champion.json", cookies=user_cookie)
champ_json = champ_dd.json()
champ_name = champ_json["data"] #this creates a list of all the champions
champ_key_dict = {}
for i in champ_name:
    champ_key_dict.update({champ_json["data"][i]["key"]:i})
#pprint.pprint(champ_dd.json())


#Professional Non-LPL data #########################################################################
#returns json with full match data
def get_match_data(url, test):
    good_url = url_for_request_scraping(url)
    json_file = requests.get(good_url, cookies=user_cookie)
    assert json_file.status_code == 200
    json_content = json_file.json()
    if test == True:
        pprint.pprint(json_content)
    return(json_content)

#returns dataframe
def build_dataframe(input_match_data, test):
    json_content = input_match_data
    count = 0
    while count < 10:
        player = json_content["participantIdentities"][count]["player"]["summonerName"]
        champion_id = json_content["participants"][count]["championId"]
        champion_name = champ_key_dict[str(champion_id)]
        gameId = json_content["gameId"]
        try:
            patch = re.match(reg, json_content["gameVersion"]).group()
        except:
            patch = "Unrecorded"
        stats = json_content["participants"][count]["stats"]
        stats.update({"championId": champion_name , "gameId": gameId, "summonerName":player, "gameVersion":patch})
        if count == 0:
            df = pd.DataFrame([stats])
            count = count + 1
        else:
            new_df = pd.DataFrame([stats])
            count = count + 1
            df = df.append(new_df) 
    if test == True:
        pprint.pprint(df)
    return(df)

on_Laptop = os.path.exists(r'C:/Users/samsl/') #Checks if we're on my laptop or Desktop
#These are all of the Desktop Files
if on_Laptop == False:
    desktop_url_file = r"C:/Users/sam/Desktop/ScrapeTest/test/test_url_file.csv"
    urllist = get_urllist(desktop_url_file)
    test_match_file = r'C:/Users/sam/Desktop/ScrapeTest/test/test_match_file_V3.csv'
    test_database_file = r'C:/Users/sam/Desktop/ScrapeTest/test/test_database_V3.csv'
    lpl_test_match_file = r'C:/Users/sam/Desktop/ScrapeTest/test/lpl_test_match_file_V3.csv'
    lpl_test_database_file = r'C:/Users/sam/Desktop/ScrapeTest/test/lpl_test_database_V3.csv'
    error_file = r'C:/Users/sam/Desktop/ScrapeTest/test/error_urls.csv'
    error_directory = r'C:/Users/sam/Desktop/ScrapeTest/test/ErrorLog'
#These are all of the Laptop files
else: 
    laptop_url_file = r'C:/Users/samsl/Desktop/ScrapeTest/test/test_url_file.csv'
    urllist = get_urllist(laptop_url_file)
    test_match_file = r'C:/Users/samsl/Desktop/ScrapeTest/test/test_match_file_V3.csv'
    test_database_file = r'C:/Users/samsl/Desktop/ScrapeTest/test/test_database_V3.csv'
    lpl_test_match_file = r'C:/Users/samsl/Desktop/ScrapeTest/test/lpl_test_match_file_V3.csv'
    lpl_test_database_file = r'C:/Users/samsl/Desktop/ScrapeTest/test/lpl_test_database_V3.csv'
    error_file = r'C:/Users/samsl/Desktop/ScrapeTest/test/error_log/error_urls.csv' 
    error_directory = r'C:/Users/samsl/Desktop/ScrapeTest/test/error_log/'
iteration_count = 0
lpl_iteration_count = 0
bad_urllist = []
start_time = time.time()
for url in urllist:
    x = get_match_data(url, False)
    print(url)
    build_dataframe(x, False)

13
http://matchhistory.na.leagueoflegends.com/en/#match-details/TRLH1/1000050045?gameHash=0fc0e7cb7e2f20c0&amp;amp;tab=overview&amp;tab=overview


AssertionError: 